In [34]:
from requests import get
from bs4 import BeautifulSoup
import os
import pandas as pd

- The goal is to automate the process to get the top 25 repos for 4 different languages 

In [138]:
# option for period is 
# daily, weekly, monthly

# enhancing the function to take a list of language and timeline as input and return lists of url and
# and a list of language

def get_top_repo(language_list, timeline):
    repo_links = []
    language=[]
    for element in language_list:
        
        url = f'https://github.com/trending/{element}?since={timeline}&spoken_language_code=en'
        response = get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        h1s = soup.find_all('h1', class_ = 'h3 lh-condensed')
        
        
        for elem  in h1s:
            repo_name = elem.get_text()
            repo_name = repo_name.replace('\n', '')
            repo_name = repo_name.replace(' ', '')
            links = 'https://github.com/' + repo_name
            repo_links.append(links)
            item= element
            language.append(item)
    df = pd.DataFrame(repo_links, language).reset_index().rename(columns = {'index': 'language',
                                                                           0:'link'})
            
        
    return df
    

In [139]:
# lets see of our function works

df1= get_top_repo(['python','swift', 'java', 'javascript'], 'weekly')

In [186]:
df1.head()

,language,link
0,python,https://github.com/microsoft/Bringing-Old-Phot...
1,python,https://github.com/corpnewt/gibMacOS
2,python,https://github.com/Rudrabha/Wav2Lip
3,python,https://github.com/PyTorchLightning/pytorch-li...
4,python,https://github.com/ytdl-org/youtube-dl


In [187]:
# getting the list of links
links = list(df1.link)
links

['https://github.com/microsoft/Bringing-Old-Photos-Back-to-Life',
 'https://github.com/corpnewt/gibMacOS',
 'https://github.com/Rudrabha/Wav2Lip',
 'https://github.com/PyTorchLightning/pytorch-lightning',
 'https://github.com/ytdl-org/youtube-dl',
 'https://github.com/Asabeneh/30-Days-Of-Python',
 'https://github.com/natewong1313/bird-bot',
 'https://github.com/public-apis/public-apis',
 'https://github.com/munki/macadmin-scripts',
 'https://github.com/facebookresearch/pifuhd',
 'https://github.com/kholia/OSX-KVM',
 'https://github.com/Hari-Nagarajan/nvidia-bot',
 'https://github.com/numpy/numpy',
 'https://github.com/mitre/caldera',
 'https://github.com/tensorflow/models',
 'https://github.com/alirezamika/autoscraper',
 'https://github.com/trekhleb/learn-python',
 'https://github.com/microsoft/DeepSpeed',
 'https://github.com/ultralytics/yolov5',
 'https://github.com/danielgatis/rembg',
 'https://github.com/eriklindernoren/PyTorch-GAN',
 'https://github.com/MrMimic/data-scientist-road

In [143]:
# creating a function that automates the parsing process given a list of links to parse

def get_content_df(links):
    '''Takes in a list of urs and parses through everyone of them and get the content of 
    that we need'''
    
    content = []
    for elem in links:
        url = elem
        response = get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        article = soup.find('article', itemprop = 'text')
        article_text = article.get_text()
        item = {
        'content': article_text
        }
        content.append(item)
        df = pd.DataFrame(content)
    return df

    

In [144]:
# checking if our function works
df = get_content_df(links)
df

,content
0,Old Photo Restoration (Official PyTorch Implem...
1,Py2/py3 script that can download macOS compone...
2,Wav2Lip: Accurately Lip-syncing Videos In The ...
3,\n\nThe lightweight PyTorch wrapper for high-p...
4,\nyoutube-dl - download videos from youtube.co...
...,...
95,\n\n\n\n\nreNgine\n\n\n\n\n\n\n\n\n\n An au...
96,🏛️ 22120 \n🏛️ - An archivist browser contro...
97,\n\n\nMaterial-UI\n\nReact components for fast...
98,"\n\n\n\n\nBootstrap\n\n Sleek, intuitive, and..."


In [145]:
# creating a final df 
final_df = pd.concat([df1, df], axis = 1)

In [154]:
# checking our final df
final_df

,language,link,content
0,python,https://github.com/microsoft/Bringing-Old-Phot...,Old Photo Restoration (Official PyTorch Implem...
1,python,https://github.com/corpnewt/gibMacOS,Py2/py3 script that can download macOS compone...
2,python,https://github.com/Rudrabha/Wav2Lip,Wav2Lip: Accurately Lip-syncing Videos In The ...
3,python,https://github.com/PyTorchLightning/pytorch-li...,\n\nThe lightweight PyTorch wrapper for high-p...
4,python,https://github.com/ytdl-org/youtube-dl,\nyoutube-dl - download videos from youtube.co...
...,...,...,...
95,javascript,https://github.com/yogeshojha/rengine,\n\n\n\n\nreNgine\n\n\n\n\n\n\n\n\n\n An au...
96,javascript,https://github.com/c9fe/22120,🏛️ 22120 \n🏛️ - An archivist browser contro...
97,javascript,https://github.com/mui-org/material-ui,\n\n\nMaterial-UI\n\nReact components for fast...
98,javascript,https://github.com/twbs/bootstrap,"\n\n\n\n\nBootstrap\n\n Sleek, intuitive, and..."


In [155]:
import unicodedata
import re
import json

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

import pandas as pd



In [166]:
def clean_content(content_list):
    clean_content = []
    for item in content_list:
        article = unicodedata.normalize('NFKD', item)\
        .encode('ascii', 'ignore')\
        .decode('utf-8', 'ignore')
        article = article.replace('\n', '')
        article = article.lower()
        clean_content.append(article)
    return clean_content
    

In [167]:
content_list = list(final_df.content)

clean = clean_content(content_list)

In [168]:
final_df['clean_content'] = clean

In [169]:
final_df

,language,link,content,clean_content
0,python,https://github.com/microsoft/Bringing-Old-Phot...,Old Photo Restoration (Official PyTorch Implem...,old photo restoration (official pytorch implem...
1,python,https://github.com/corpnewt/gibMacOS,Py2/py3 script that can download macOS compone...,py2/py3 script that can download macos compone...
2,python,https://github.com/Rudrabha/Wav2Lip,Wav2Lip: Accurately Lip-syncing Videos In The ...,wav2lip: accurately lip-syncing videos in the ...
3,python,https://github.com/PyTorchLightning/pytorch-li...,\n\nThe lightweight PyTorch wrapper for high-p...,the lightweight pytorch wrapper for high-perfo...
4,python,https://github.com/ytdl-org/youtube-dl,\nyoutube-dl - download videos from youtube.co...,youtube-dl - download videos from youtube.com ...
...,...,...,...,...
95,javascript,https://github.com/yogeshojha/rengine,\n\n\n\n\nreNgine\n\n\n\n\n\n\n\n\n\n An au...,rengine an automated recon framework for we...
96,javascript,https://github.com/c9fe/22120,🏛️ 22120 \n🏛️ - An archivist browser contro...,22120 - an archivist browser controller t...
97,javascript,https://github.com/mui-org/material-ui,\n\n\nMaterial-UI\n\nReact components for fast...,material-uireact components for faster and sim...
98,javascript,https://github.com/twbs/bootstrap,"\n\n\n\n\nBootstrap\n\n Sleek, intuitive, and...","bootstrap sleek, intuitive, and powerful fron..."


In [243]:
def get_content_df(links):
    '''Takes in a list of urs and parses through everyone of them and get the content of     that we need'''
    content = []
    watchers = []
    stars = []
    forks = []
    
    for elem in links:
        counts = []
        url = elem    
        response = get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        article = soup.find('article', itemprop = 'text')
        article_text = article.get_text()
        item = {        'content': article_text
               }        
        content.append(item)
        text = soup.find_all('a', class_='social-count')
        # so for every h in text within an element, we
        for h in text:
            #['watchers','stars','forks']
            counts.append(h.get_text().replace('\n','').replace(' ',''))
        
        watchers.append(counts[0])
        stars.append(counts[1])
        forks.append(counts[2])
   
  

    df = pd.DataFrame(content)
    df['watchers'] = watchers  
    df['stars'] = stars
    df['forks'] = forks        
    return df

In [247]:
# df = get_content_df(links)

In [248]:
df

,content,watchers,stars,forks
0,Old Photo Restoration (Official PyTorch Implem...,169,4.2k,277
1,Py2/py3 script that can download macOS compone...,109,2.6k,356
2,Wav2Lip: Accurately Lip-syncing Videos In The ...,59,1.6k,254
3,\n\nThe lightweight PyTorch wrapper for high-p...,215,10.3k,1.2k
4,\nyoutube-dl - download videos from youtube.co...,2.1k,79.8k,1.6k
...,...,...,...,...
95,\n\n\n\n\nreNgine\n\n\n\n\n\n\n\n\n\n An au...,93,2.1k,299
96,🏛️ 22120 \n🏛️ - An archivist browser contro...,35,2.2k,75
97,\n\n\nMaterial-UI\n\nReact components for fast...,1.4k,62.6k,19.1k
98,"\n\n\n\n\nBootstrap\n\n Sleek, intuitive, and...",7.1k,145k,71.1k


In [251]:
# final_df.drop(columns = 'content', inplace = True)

In [252]:

# df1 = pd.concat([df,final_df], axis = 1)

,content,watchers,stars,forks,language,link,clean_content
0,Old Photo Restoration (Official PyTorch Implem...,169,4.2k,277,python,https://github.com/microsoft/Bringing-Old-Phot...,old photo restoration (official pytorch implem...
1,Py2/py3 script that can download macOS compone...,109,2.6k,356,python,https://github.com/corpnewt/gibMacOS,py2/py3 script that can download macos compone...
2,Wav2Lip: Accurately Lip-syncing Videos In The ...,59,1.6k,254,python,https://github.com/Rudrabha/Wav2Lip,wav2lip: accurately lip-syncing videos in the ...
3,\n\nThe lightweight PyTorch wrapper for high-p...,215,10.3k,1.2k,python,https://github.com/PyTorchLightning/pytorch-li...,the lightweight pytorch wrapper for high-perfo...
4,\nyoutube-dl - download videos from youtube.co...,2.1k,79.8k,1.6k,python,https://github.com/ytdl-org/youtube-dl,youtube-dl - download videos from youtube.com ...
...,...,...,...,...,...,...,...
95,\n\n\n\n\nreNgine\n\n\n\n\n\n\n\n\n\n An au...,93,2.1k,299,javascript,https://github.com/yogeshojha/rengine,rengine an automated recon framework for we...
96,🏛️ 22120 \n🏛️ - An archivist browser contro...,35,2.2k,75,javascript,https://github.com/c9fe/22120,22120 - an archivist browser controller t...
97,\n\n\nMaterial-UI\n\nReact components for fast...,1.4k,62.6k,19.1k,javascript,https://github.com/mui-org/material-ui,material-uireact components for faster and sim...
98,"\n\n\n\n\nBootstrap\n\n Sleek, intuitive, and...",7.1k,145k,71.1k,javascript,https://github.com/twbs/bootstrap,"bootstrap sleek, intuitive, and powerful fron..."


In [254]:
# df1.to_csv('nlp-data.csv')

In [255]:
df1

,content,watchers,stars,forks,language,link,clean_content
0,Old Photo Restoration (Official PyTorch Implem...,169,4.2k,277,python,https://github.com/microsoft/Bringing-Old-Phot...,old photo restoration (official pytorch implem...
1,Py2/py3 script that can download macOS compone...,109,2.6k,356,python,https://github.com/corpnewt/gibMacOS,py2/py3 script that can download macos compone...
2,Wav2Lip: Accurately Lip-syncing Videos In The ...,59,1.6k,254,python,https://github.com/Rudrabha/Wav2Lip,wav2lip: accurately lip-syncing videos in the ...
3,\n\nThe lightweight PyTorch wrapper for high-p...,215,10.3k,1.2k,python,https://github.com/PyTorchLightning/pytorch-li...,the lightweight pytorch wrapper for high-perfo...
4,\nyoutube-dl - download videos from youtube.co...,2.1k,79.8k,1.6k,python,https://github.com/ytdl-org/youtube-dl,youtube-dl - download videos from youtube.com ...
...,...,...,...,...,...,...,...
95,\n\n\n\n\nreNgine\n\n\n\n\n\n\n\n\n\n An au...,93,2.1k,299,javascript,https://github.com/yogeshojha/rengine,rengine an automated recon framework for we...
96,🏛️ 22120 \n🏛️ - An archivist browser contro...,35,2.2k,75,javascript,https://github.com/c9fe/22120,22120 - an archivist browser controller t...
97,\n\n\nMaterial-UI\n\nReact components for fast...,1.4k,62.6k,19.1k,javascript,https://github.com/mui-org/material-ui,material-uireact components for faster and sim...
98,"\n\n\n\n\nBootstrap\n\n Sleek, intuitive, and...",7.1k,145k,71.1k,javascript,https://github.com/twbs/bootstrap,"bootstrap sleek, intuitive, and powerful fron..."


## Prepare Phase

In [256]:
def clean_content(content_list):
    clean_content = []
    for item in content_list:
        article = unicodedata.normalize('NFKD', item)\
        .encode('ascii', 'ignore')\
        .decode('utf-8', 'ignore')
        article = article.replace('\n', '')
        article = article.lower()
        article = re.sub(r'[^a-z0-9\s]', '', article)
        clean_content.append(article)
    return clean_content

In [257]:
def tokenizing(content_list):
    tokenize_content = []
    tokenizer = nltk.tokenize.ToktokTokenizer()
    for item in content_list:
        article = tokenizer.tokenize(item, return_str = True)
        tokenize_content.append(article)
    return tokenize_content
        
    

In [268]:
content_list = df1.content


In [269]:
df1['clean'] = clean_content(content_list)

In [270]:
df1['tokenized'] = tokenizing(df1.clean)

In [271]:
df1

,content,watchers,stars,forks,language,link,clean_content,clean,tokenized
0,Old Photo Restoration (Official PyTorch Implem...,169,4.2k,277,python,https://github.com/microsoft/Bringing-Old-Phot...,old photo restoration (official pytorch implem...,old photo restoration official pytorch impleme...,old photo restoration official pytorch impleme...
1,Py2/py3 script that can download macOS compone...,109,2.6k,356,python,https://github.com/corpnewt/gibMacOS,py2/py3 script that can download macos compone...,py2py3 script that can download macos componen...,py2py3 script that can download macos componen...
2,Wav2Lip: Accurately Lip-syncing Videos In The ...,59,1.6k,254,python,https://github.com/Rudrabha/Wav2Lip,wav2lip: accurately lip-syncing videos in the ...,wav2lip accurately lipsyncing videos in the wi...,wav2lip accurately lipsyncing videos in the wi...
3,\n\nThe lightweight PyTorch wrapper for high-p...,215,10.3k,1.2k,python,https://github.com/PyTorchLightning/pytorch-li...,the lightweight pytorch wrapper for high-perfo...,the lightweight pytorch wrapper for highperfor...,the lightweight pytorch wrapper for highperfor...
4,\nyoutube-dl - download videos from youtube.co...,2.1k,79.8k,1.6k,python,https://github.com/ytdl-org/youtube-dl,youtube-dl - download videos from youtube.com ...,youtubedl download videos from youtubecom or ...,youtubedl download videos from youtubecom or o...
...,...,...,...,...,...,...,...,...,...
95,\n\n\n\n\nreNgine\n\n\n\n\n\n\n\n\n\n An au...,93,2.1k,299,javascript,https://github.com/yogeshojha/rengine,rengine an automated recon framework for we...,rengine an automated recon framework for we...,rengine an automated recon framework for web a...
96,🏛️ 22120 \n🏛️ - An archivist browser contro...,35,2.2k,75,javascript,https://github.com/c9fe/22120,22120 - an archivist browser controller t...,22120 an archivist browser controller th...,22120 an archivist browser controller that cac...
97,\n\n\nMaterial-UI\n\nReact components for fast...,1.4k,62.6k,19.1k,javascript,https://github.com/mui-org/material-ui,material-uireact components for faster and sim...,materialuireact components for faster and simp...,materialuireact components for faster and simp...
98,"\n\n\n\n\nBootstrap\n\n Sleek, intuitive, and...",7.1k,145k,71.1k,javascript,https://github.com/twbs/bootstrap,"bootstrap sleek, intuitive, and powerful fron...",bootstrap sleek intuitive and powerful fronte...,bootstrap sleek intuitive and powerful fronten...


In [278]:
def stemming(content_list):
    stemmed = []
    ps = nltk.porter.PorterStemmer()
    for item in content_list:
        stems = [ps.stem(words) for words in item.split()]
        item_stemmed = ' '.join(stems)
        stemmed.append(item_stemmed)
    return stemmed

In [279]:
stemmed = stemming(df1.tokenized)

In [280]:
len(stemmed)

100

In [281]:
df1['stemmed'] = stemmed

In [311]:
df1.head()

,content,watchers,stars,forks,language,link,clean_content,clean,tokenized,stemmed,lemmatized,filtered
0,Old Photo Restoration (Official PyTorch Implem...,169,4.2k,277,python,https://github.com/microsoft/Bringing-Old-Phot...,old photo restoration (official pytorch implem...,old photo restoration official pytorch impleme...,old photo restoration official pytorch impleme...,old photo restor offici pytorch implementation...,old photo restoration official pytorch impleme...,old photo restoration official pytorch impleme...
1,Py2/py3 script that can download macOS compone...,109,2.6k,356,python,https://github.com/corpnewt/gibMacOS,py2/py3 script that can download macos compone...,py2py3 script that can download macos componen...,py2py3 script that can download macos componen...,py2py3 script that can download maco compon di...,py2py3 script that can download macos componen...,py2py3 script download macos component direct ...
2,Wav2Lip: Accurately Lip-syncing Videos In The ...,59,1.6k,254,python,https://github.com/Rudrabha/Wav2Lip,wav2lip: accurately lip-syncing videos in the ...,wav2lip accurately lipsyncing videos in the wi...,wav2lip accurately lipsyncing videos in the wi...,wav2lip accur lipsync video in the wildthi cod...,wav2lip accurately lipsyncing video in the wil...,wav2lip accurately lipsyncing video wildthis c...
3,\n\nThe lightweight PyTorch wrapper for high-p...,215,10.3k,1.2k,python,https://github.com/PyTorchLightning/pytorch-li...,the lightweight pytorch wrapper for high-perfo...,the lightweight pytorch wrapper for highperfor...,the lightweight pytorch wrapper for highperfor...,the lightweight pytorch wrapper for highperfor...,the lightweight pytorch wrapper for highperfor...,lightweight pytorch wrapper highperformance ai...
4,\nyoutube-dl - download videos from youtube.co...,2.1k,79.8k,1.6k,python,https://github.com/ytdl-org/youtube-dl,youtube-dl - download videos from youtube.com ...,youtubedl download videos from youtubecom or ...,youtubedl download videos from youtubecom or o...,youtubedl download video from youtubecom or ot...,youtubedl download video from youtubecom or ot...,youtubedl download video youtubecom video plat...


In [283]:
def lemmatizing(content_list):
    wnl = nltk.stem.WordNetLemmatizer()
    lemmatized = []
    for item in content_list:
        lemmas = [wnl.lemmatize(word) for word in item.split()]
        item_lemmatized = ' '.join(lemmas)
        lemmatized.append(item_lemmatized)
    return lemmatized

In [284]:
lemmatized = lemmatizing(df1.tokenized)

In [288]:
df1['lemmatized'] = lemmatized

In [312]:
df1.head()

,content,watchers,stars,forks,language,link,clean_content,clean,tokenized,stemmed,lemmatized,filtered
0,Old Photo Restoration (Official PyTorch Implem...,169,4.2k,277,python,https://github.com/microsoft/Bringing-Old-Phot...,old photo restoration (official pytorch implem...,old photo restoration official pytorch impleme...,old photo restoration official pytorch impleme...,old photo restor offici pytorch implementation...,old photo restoration official pytorch impleme...,old photo restoration official pytorch impleme...
1,Py2/py3 script that can download macOS compone...,109,2.6k,356,python,https://github.com/corpnewt/gibMacOS,py2/py3 script that can download macos compone...,py2py3 script that can download macos componen...,py2py3 script that can download macos componen...,py2py3 script that can download maco compon di...,py2py3 script that can download macos componen...,py2py3 script download macos component direct ...
2,Wav2Lip: Accurately Lip-syncing Videos In The ...,59,1.6k,254,python,https://github.com/Rudrabha/Wav2Lip,wav2lip: accurately lip-syncing videos in the ...,wav2lip accurately lipsyncing videos in the wi...,wav2lip accurately lipsyncing videos in the wi...,wav2lip accur lipsync video in the wildthi cod...,wav2lip accurately lipsyncing video in the wil...,wav2lip accurately lipsyncing video wildthis c...
3,\n\nThe lightweight PyTorch wrapper for high-p...,215,10.3k,1.2k,python,https://github.com/PyTorchLightning/pytorch-li...,the lightweight pytorch wrapper for high-perfo...,the lightweight pytorch wrapper for highperfor...,the lightweight pytorch wrapper for highperfor...,the lightweight pytorch wrapper for highperfor...,the lightweight pytorch wrapper for highperfor...,lightweight pytorch wrapper highperformance ai...
4,\nyoutube-dl - download videos from youtube.co...,2.1k,79.8k,1.6k,python,https://github.com/ytdl-org/youtube-dl,youtube-dl - download videos from youtube.com ...,youtubedl download videos from youtubecom or ...,youtubedl download videos from youtubecom or o...,youtubedl download video from youtubecom or ot...,youtubedl download video from youtubecom or ot...,youtubedl download video youtubecom video plat...


In [296]:
def remove_stopwords(content_list):
    no_stopwords = []
    stopword_list = stopwords.words('english')
    for item in content_list:
        words = item.split()
        filtered_words = [w for w in words if w not in stopword_list]
        article_without_stopwords = ' '.join(filtered_words)
        no_stopwords.append(article_without_stopwords)
    return no_stopwords

In [297]:
filtered = remove_stopwords(df1.lemmatized)

In [300]:
df1['filtered'] = filtered

In [313]:
df1.head()

,content,watchers,stars,forks,language,link,clean_content,clean,tokenized,stemmed,lemmatized,filtered
0,Old Photo Restoration (Official PyTorch Implem...,169,4.2k,277,python,https://github.com/microsoft/Bringing-Old-Phot...,old photo restoration (official pytorch implem...,old photo restoration official pytorch impleme...,old photo restoration official pytorch impleme...,old photo restor offici pytorch implementation...,old photo restoration official pytorch impleme...,old photo restoration official pytorch impleme...
1,Py2/py3 script that can download macOS compone...,109,2.6k,356,python,https://github.com/corpnewt/gibMacOS,py2/py3 script that can download macos compone...,py2py3 script that can download macos componen...,py2py3 script that can download macos componen...,py2py3 script that can download maco compon di...,py2py3 script that can download macos componen...,py2py3 script download macos component direct ...
2,Wav2Lip: Accurately Lip-syncing Videos In The ...,59,1.6k,254,python,https://github.com/Rudrabha/Wav2Lip,wav2lip: accurately lip-syncing videos in the ...,wav2lip accurately lipsyncing videos in the wi...,wav2lip accurately lipsyncing videos in the wi...,wav2lip accur lipsync video in the wildthi cod...,wav2lip accurately lipsyncing video in the wil...,wav2lip accurately lipsyncing video wildthis c...
3,\n\nThe lightweight PyTorch wrapper for high-p...,215,10.3k,1.2k,python,https://github.com/PyTorchLightning/pytorch-li...,the lightweight pytorch wrapper for high-perfo...,the lightweight pytorch wrapper for highperfor...,the lightweight pytorch wrapper for highperfor...,the lightweight pytorch wrapper for highperfor...,the lightweight pytorch wrapper for highperfor...,lightweight pytorch wrapper highperformance ai...
4,\nyoutube-dl - download videos from youtube.co...,2.1k,79.8k,1.6k,python,https://github.com/ytdl-org/youtube-dl,youtube-dl - download videos from youtube.com ...,youtubedl download videos from youtubecom or ...,youtubedl download videos from youtubecom or o...,youtubedl download video from youtubecom or ot...,youtubedl download video from youtubecom or ot...,youtubedl download video youtubecom video plat...


In [310]:
# pd.Series(' '.join(df1[df1.language =='python'].filtered))